# Zoptymalizowana symulacja plazmowa Particle in Cell w Pythonie

* Dominik Stańczak
* opiekun badań: dr Sławomir Jabłoński, IFPiLM
* kierujący pracą: dr inż Daniel Kikoła

# Plazma - *po co* ją symulować?
* cała dziedzina fizyki skupiona na zastosowaniach - głównie do fuzji termojądrowej
    * ![https://en.wikipedia.org/wiki/File:Fusion_rxnrate.svg](prezentacja_sem_dyp/img/fusion_temperature_plot.png)
    * Potrzebujemy ogromnych temperatur, by ją osiągnąć - osiągalne jedynie w plazmach
    * Również inne zastosowania - akceleratory (CERNowski projekt *AWAKE*), silniki kosmiczne (Halla), obróbka procesorów, medycyna...

* Wyjątkowo złożony układ: nie znamy analitycznych rozwiązań na praktycznie żaden z ciekawszych przypadków
* Główny koncepcyjny problem: liczba dalekozasięgowych oddziaływań w układzie rośnie jak $n^2$ w liczbie cząstek

# Plazma - *jak* ją symulować?

### Równanie Vlasova - podstawowy, najściślejszy aparat (tu: w uproszczonej elektrostatycznej wersji)

$$ \frac{\partial f_\alpha}{\partial t} + \vec{v}_\alpha \cdot \frac{\partial f_\alpha}{\partial \vec{x}} + \frac{q_\alpha \vec{E}}{m_\alpha} \cdot \frac{\partial f_\alpha}{\partial \vec{v}} = 0$$

$f$ jest gęstością cząstek w danym punkcie w przestrzeni fazowej. $\alpha$ oznacza konkretny typ cząstek. $E$ to pole elektryczne, $q$ - ładunek, $m$ - masa.

### Problemy z równianiem Vlasova

W jednej chwili czasu pracujemy na 6-wymiarowej siatce. Więc problem skaluje się jak $N_x^3N_v^3$. Siatka musi mieć dobrą rozdzielczość i zasięg (zjawisko wysokoonergetycznego ogona elektronowego).

> Superkomputer Titan w Oak Ridge National Laboratory, USA ma 27 petaflopów wydajności obliczeniowej i zużywa do 9 megawatów mocy. Bez maszyny na skalę Titana sensownie nie da się czegoś takiego zrobić.

## Przybliżenia
Oczywiście pomijają część efektów, np. magnetohydrodynamika
* nie działa daleko od równowagowej dystrybucji
* wymaga, aby plazma była zdominowana przez kolizje
* zupełnie się nie nadaje do symulacji plazmy termojądrowej

Oraz są modele opierające się na fundamentalnej fizyce ruchu cząstek według równań Newtona, jak na przykład...

# Modele Particle in Cell

In [2]:
import pic3
pic3.cold_plasma_oscillations("dane1.hdf5")

Runtime: 0.3527693748474121
Saved file to dane1.hdf5


## Główny algorytm
Symulacje cząsteczkowe są stosunkowo łatwe ze względu na ich **liniowe skalowanie w liczbie cząstek**. Problematycznym krokiem jest jedynie obliczanie oddziaływań - w przypadku ogólnym, gdy każda cząstka może oddziaływać na każdą, liczba oddziaływań skaluje się jak $n^2$.

Jeśli więc (upraszczając) nasz zasób pamięć RAM wystarczy na milion cząstek, to na dwa miliony będziemy potrzebować cztery razy tyle RAMu. Milion zaś to niewiele!

### Czy musimy obliczać siły bezpośrednio?

Zdefiniujmy sobie dyskretną siatkę o liczbie komórek $n_g << n$. Każda z cząstek wpada w którąś komórkę.

Policzmy, ile jest cząstek każdego rodzaju w każdej komórce. Sumując ładunki w komórce, dostajemy gęstość ładunku. Złożoność $O(n)$ - wykonujemy tą operację raz dla każdej cząstki. To tzw. **depozycja ładunku**. *W rzeczywistości - interpolacja do krawędzi siatki!*

In [3]:
import test_scatter
%matplotlib qt5
test_scatter.test_constant_density(plotting=True)

charge density [ 16.  16.  16.  16.  16.  16.  16.  16.]


In [4]:
%matplotlib qt5
test_scatter.test_single_particle(plotting=True)

[3 5]
charge density [ 0.    0.    0.    0.5   0.5   0.25  0.75  0.  ]


Mamy więc na $n_g$ komórek siatki zdefiniowaną gęstość ładunku. Możemy teraz wykorzystać równanie Poissona:

$$\frac{\rho}{\varepsilon_0} = \nabla E = -\nabla^2 \varphi$$

Ze znanym $\rho$ można wykorzystać metody numeryczne jak

* Jacobiego
* Gaussa-Seidela
* Successive Over-relaxation
* Conjugate Gradients
* **metody spektralne**, wykorzystujące transformaty Fouriera *(algorytm FFT w przypadkach dyskretnych!)*

aby obliczyć $E$ - ze złożonością zazwyczaj faktycznie kwadratową *(dowód trywialny)*, ale w $n_g$ - zaś jeśli $n_g << n$, to $n_g^2 << n^2$! **Koszt obliczenia rzędu $n_g^2$ nie jest taki straszny!**

In [5]:
import test_FourierSolver
%matplotlib qt5
test_FourierSolver.test_PoissonSolver(debug=True)

/home/dominik/.anaconda3/lib/python3.5/site-packages/matplotlib/axes/_axes.py:531: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [6]:
%matplotlib qt5
test_FourierSolver.test_PoissonSolver_sheets(debug=True)

/home/dominik/.anaconda3/lib/python3.5/site-packages/matplotlib/axes/_axes.py:531: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


### Mamy więc obliczone pole elektryczne - ale na siatce. Co z tego?

Tyle, iż teraz możemy zinterpolować wartości pola z siatki do cząstek, raz na cząstkę - $O(n)$, wciąż mniej niż kwadratowo!

In [7]:
import test_gather
%matplotlib qt5
test_gather.test_single_particle(plotting=True)

L2 norm:  4.14514438616e-06


In [8]:
%matplotlib qt5
test_gather.test_periodic(plotting=True)

L2 norm:  0.000201842971824
L2 norm:  0.000201842971824


### Mamy więc siły działające na cząstki

Teraz zaś już trywialną kwestią jest użyć ich oraz znanych uprzednio pędów, by zaktualizować pędy i położenia cząstek do kolejnej iteracji. (tzw. **particle push**)

In [9]:
import test_pusher
%matplotlib qt5
test_pusher.test_constant_field()

[ 0.]
[ 0.      -0.00125 -0.0025  -0.00375 -0.005   -0.00625 -0.0075  -0.00875
 -0.01    -0.01125 -0.0125  -0.01375 -0.015   -0.01625 -0.0175  -0.01875
 -0.02    -0.02125 -0.0225  -0.02375 -0.025   -0.02625 -0.0275  -0.02875
 -0.03    -0.03125 -0.0325  -0.03375 -0.035   -0.03625 -0.0375  -0.03875
 -0.04    -0.04125 -0.0425  -0.04375 -0.045   -0.04625 -0.0475  -0.04875
 -0.05    -0.05125 -0.0525  -0.05375 -0.055   -0.05625 -0.0575  -0.05875
 -0.06    -0.06125 -0.0625  -0.06375 -0.065   -0.06625 -0.0675  -0.06875
 -0.07    -0.07125 -0.0725  -0.07375 -0.075   -0.07625 -0.0775  -0.07875
 -0.08    -0.08125 -0.0825  -0.08375 -0.085   -0.08625 -0.0875  -0.08875
 -0.09    -0.09125 -0.0925  -0.09375 -0.095   -0.09625 -0.0975  -0.09875
 -0.1     -0.10125 -0.1025  -0.10375 -0.105   -0.10625 -0.1075  -0.10875
 -0.11    -0.11125 -0.1125  -0.11375 -0.115   -0.11625 -0.1175  -0.11875
 -0.12    -0.12125 -0.1225  -0.12375 -0.125   -0.12625 -0.1275  -0.12875
 -0.13    -0.13125 -0.1325  -0.13375 -0.135  

Przesunięte na nowo cząstki można zaś wykorzystać do znalezienia nowego rozkładu ładunku... I wracamy do punktu wyjścia!

# Prawdopodobnie najważniejszy fakt czyniący symulacje PIC sensownymi:

W równaniach ruchu oddziaływania elektryczne skalują się jak $\frac{q}{m} = \frac{Nq}{Nm}$. Jedna cząstka obliczeniowa może reprezentować wiele cząstek rzeczywistych! To pozwala nam symulować plazmę makroskalową!

# Stan obecny prac

* Korzystam jak dotąd głównie z książki Birdsalla & Langdona z 1975, *Plasma Physics via Computer Simulation* oraz materiałów (m. in. wykładów) dostępnych w Internecie
* Docelowo ma być elektromagnetyczna i symulować oddziaływanie plazmy z laserem, więc przypadek relatywistyczny 
* Model na tym etapie jest elektrostatyczny, okresowy (nieskończona plazma), nierelatywistyczny (pracuję na prędkościach i przyjmuję $\gamma = 1$)
* Symulacja działa dobrze dla klasycznego problemu oscylacji zimnej plazmy

In [10]:
from pic3 import cold_plasma_oscillations
cold_plasma_oscillations("1default.hdf5")
cold_plasma_oscillations("2dense.hdf5", N_electrons=2**9+1)
cold_plasma_oscillations("3leapfrog_instability_pre.hdf5", dt = 0.2, NT=150, N_electrons=128, NG=32)
cold_plasma_oscillations("4leapfrog_instability_post.hdf5", dt = 3, NT=150, N_electrons=2**9+1, NG=32)
cold_plasma_oscillations("5wavebreaking.hdf5", dt = 0.2, NT=150, N_electrons=2**11+1, NG=32, push_amplitude=2)
cold_plasma_oscillations("6aliasing.hdf5", dt = 0.2, NT=150, N_electrons=2**9+1, NG=32, push_mode=18)

Runtime: 0.4331374168395996
Saved file to 1default.hdf5
Runtime: 0.3853340148925781
Saved file to 2dense.hdf5
Runtime: 0.307403564453125
Saved file to 3leapfrog_instability_pre.hdf5
Runtime: 0.3516044616699219
Saved file to 4leapfrog_instability_post.hdf5
Runtime: 0.9237394332885742
Saved file to 5wavebreaking.hdf5
Runtime: 0.3480665683746338
Saved file to 6aliasing.hdf5


In [ ]:
%%bash
python plotting.py -dev -lines 1default.hdf5

# Specyfika symulacji
* Jeden wymiar, ale jest napisana tak, by łatwo było ją rozbudować na więcej
* *field solver* jest obecnie spektralny, wykorzystuje dane z całej siatki (globalnie). *To się kłóci z relatywistyką!*
* *Particle pusher* to tzw. algorytm Borysa, podobny do algorytmu Verleta - w przypadku jednowymiarowym elektrostatycznym redukuje się do klasycznego *leapfroga*.
* Obecnie: okresowe warunki brzegowe, cząstki wychodzące z prawej wchodzą od lewej.

# Problemy, które do tej pory napotkałem
* Zaczynanie kodu od początku zamiast rozsądnej przeróbki tego, co mam
* Mnóstwo błędów wynikających z **nieprzemyślanego dostępu do zmiennych**
* Obliczanie pola oraz energii - **skalowanie**, problem utworzenia wektora $k$

# Co pozostało do zrobienia:

* dobudować pole magnetyczne w pusherze oraz field solverze
* przejść na solver i pusher relatywistyczny
    * lokalny! Wrócimy do tego wątku później...
* Dodać laser (jako warunek brzegowy zależny od czasu na pola $E$, $B$)
* Testy prędkości i optymalizacja - próby wyciągnięcia maksymalnej możliwej prędkości

# Optymalizacja kodu
### Numpy
* Python jest uznawany za "wolny" - ale C i Fortran nie są.
* Python jest wybitnym językiem "klejącym". Odwołując się z poziomu Pythona do gotowych implementacji operacji wektorowych w innych architekturach, możemy osiągnąć szybkości praktycznie takie same jak w językach niskopoziomowych
* Standardową biblioteką całego *scientific computing* w Pythonie jest Numpy, implementujący operacje wektorowe

In [11]:
import numpy as np

N = 1000
x = np.random.random(N)

def odleglosci_sekwencyjnie(x):
    r = np.zeros((N, N))
    for i in range(N):
        for j in range(i, N):
            d = x[i] - x[j]
            r[i, j] = d
            r[j, i] = -d
    return r

odleglosci_sekwencyjnie(x)[:4,:4]

array([[-0.        , -0.44081255, -0.14142763, -0.07701312],
       [ 0.44081255, -0.        ,  0.29938492,  0.36379943],
       [ 0.14142763, -0.29938492, -0.        ,  0.0644145 ],
       [ 0.07701312, -0.36379943, -0.0644145 , -0.        ]])

In [12]:
def odleglosci_numpy(x):
    return x.reshape(N, 1) - x.reshape(1, N)

odleglosci_numpy(x)[:4,:4]

array([[ 0.        , -0.44081255, -0.14142763, -0.07701312],
       [ 0.44081255,  0.        ,  0.29938492,  0.36379943],
       [ 0.14142763, -0.29938492,  0.        ,  0.0644145 ],
       [ 0.07701312, -0.36379943, -0.0644145 ,  0.        ]])

In [13]:
%timeit odleglosci_sekwencyjnie(x)

1 loop, best of 3: 3.64 s per loop


In [14]:
%timeit odleglosci_numpy(x)

10 loops, best of 3: 12.7 ms per loop


### Numba

Kompilacja *just-in-time* sekwencyjnego kodu Pythona do języków niskopoziomowych i odwoływanie się do takiej, zapisanej implementacji

In [15]:
from numba import njit

@njit() # <- tak, to jest jedyna różnica
def odleglosci_sekwencyjnie_numba(x):
    r = np.zeros((N, N))
    for i in range(N):
        for j in range(i, N):
            d = x[i] - x[j]
            r[i, j] = d
            r[j, i] = -d
    return r

odleglosci_sekwencyjnie_numba(x)[:4,:4]

array([[-0.        , -0.44081255, -0.14142763, -0.07701312],
       [ 0.44081255, -0.        ,  0.29938492,  0.36379943],
       [ 0.14142763, -0.29938492, -0.        ,  0.0644145 ],
       [ 0.07701312, -0.36379943, -0.0644145 , -0.        ]])

In [16]:
%timeit odleglosci_sekwencyjnie_numba(x)

The slowest run took 4.03 times longer than the fastest. This could mean that an intermediate result is being cached.
10 loops, best of 3: 26.3 ms per loop


### Dygresja teoretyczna - paralelizm w obliczeniach
* Problemy trywialnie paralelizowalne
* Złożone podejścia do paralelizmu
* Karty graficzne!
* Kopiowanie pamięci

## PyCUDA

PIC jest dlatego przyszłościowy, że jest *trywialnie paralelizowalny* - każda z cząstek jest niezależna od siebie, jedynie od siatki i pola.

Istnieją implementacje obliczeń macierzowych z Numpy wykorzystujące karty graficzne Nvidii.

Numpy osiąga paralelizm na procesorze (przykładowo, 4 rdzenie dają 4 wątki) - przykładowa starzejąca się karta Nvidii pozwala uruchomić 1024 wątki.

PIC jest więc *cudownym* przykładem programu do uruchomienia kart graficznych!

Relatywistyczny *field solver* jest zazwyczaj lokalny - to też pozwala uruchomić go na karcie graficznej!